In [68]:
a_file = open("refe.txt", "r")
def cloning(li):
    new_li=[]
    for item in li:
        new_li.append(item)
    return new_li    

layer1 = []
layer2 = []


for line in a_file:
    stripped_line = line.strip()
    #print(stripped_line)
    line_list = stripped_line.split()
    if line_list[-1]!='<eos>':
        layer2.append(line_list)
    else:
        layer2.append(line_list[:len(line_list)-1])
        
    #print(layer2)
    if line_list[-1]=='<eos>':
       
        layer1.append(cloning(layer2))
        print(layer1)
        layer2.clear()
        #print(layer1)
        
#print(layer1)       

#layer1 ->[[['I', 'want', 'food'], ['I', 'want', 'the', 'food', '<eos>']]]
#layer1 ->[[]] ->[layer2]
#layer2 ->[['I', 'came', '<eos>']]
#layer1 ->[[['I', 'came', '<eos>']]]
#layer1 ->[[],[]]
#

[[['Good', 'morning.']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']], [['Look', 'at', 'me', 'singing!']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']], [['Look', 'at', 'me', 'singing!']], [['It', 'is', '9', 'in', 'the', 'morning.']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']], [['Look', 'at', 'me', 'singing!']], [['It', 'is', '9', 'in', 'the', 'morning.']], [['I', 'want', 'food.'], ['I', 'am', 'hungry.']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']], [['Look', 'at', 'me', 'singing!']], [['It', 'is', '9', 'in', 'the', 'morning.']], [['I', 'want', 'food.'], ['I', 'am', 'hungry.']], [['I', 'am', 'understanding.']]]
[[['Good', 'morning.']], [['Where', 'is', 'the', 'bank?']], [['Look', 'at', 'me', 'singing!']], [['It', 'is', '9', 'in', 'the', 'morning.']], [['I', 'want', 'food.'], ['I', 'am', 'hungry.']], [['I', 'am', 'understanding.']], [['I', 'understood.']]]
[[['Good', 

In [69]:
b_file = open("predict.txt", "r")


list_of_lists_ = []

for line in b_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    list_of_lists_.append(line_list)


b_file.close()


print(list_of_lists_)

[['it', 'is', 'in', 'the', 'afternoon'], ['where', 'is', 'the', 'bank'], ['look', 'at', 'me', 'singing', '!'], ['i', 'am', 'hurt'], ['i', 'want', 'food'], ['i', 'understand'], ['i', 'do', 'not', 'understand'], ['i', 'was', 'painter'], ['where', 'are', 'you', 'going', 'today?'], ['i', 'like', 'blue'], ['i', 'am', 'really', 'woried', 'because', 'of', 'rain'], ['i', 'have', 'to', 'me'], ['i', 'am', 'sorry'], ['i', 'am', 'engineer'], ['wait', 'i', 'am', 'thinking'], ['please', 'call', 'an', 'ambulance'], ['no', 'smoking', 'please'], ['will', 'he', 'go', 'to', 'school'], ['i', 'havenâ€˜t', 'had', 'tea', 'yet']]


In [70]:
import collections
import math



def _get_ngrams(segment, max_order):
  """Extracts all n-grams upto a given maximum order from an input segment.
  Args:
    segment: text segment from which n-grams will be extracted.
    max_order: maximum length in tokens of the n-grams returned by this
        methods.
  Returns:
    The Counter containing all n-grams upto max_order in segment
    with a count of how many times each n-gram occurred.
  """
  ngram_counts = collections.Counter()
  for order in range(1, max_order + 1):
    for i in range(0, len(segment) - order + 1):
      ngram = tuple(segment[i:i+order])
      ngram_counts[ngram] += 1
  return ngram_counts


def compute_bleu(reference_corpus, translation_corpus, max_order=1,
                 smooth=False):
  """Computes BLEU score of translated segments against one or more references.
  Args:
    reference_corpus: list of lists of references for each translation. Each
        reference should be tokenized into a list of tokens.
    translation_corpus: list of translations to score. Each translation
        should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
  Returns:
    3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
    precisions and brevity penalty.
  """
  matches_by_order = [0] * max_order
  possible_matches_by_order = [0] * max_order
  reference_length = 0
  translation_length = 0
  for (references, translation) in zip(reference_corpus,
                                       translation_corpus):
    reference_length += min(len(r) for r in references)
    translation_length += len(translation)

    merged_ref_ngram_counts = collections.Counter()
    for reference in references:
      merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
    translation_ngram_counts = _get_ngrams(translation, max_order)
    overlap = translation_ngram_counts & merged_ref_ngram_counts
    for ngram in overlap:
      matches_by_order[len(ngram)-1] += overlap[ngram]
    for order in range(1, max_order+1):
      possible_matches = len(translation) - order + 1
      if possible_matches > 0:
        possible_matches_by_order[order-1] += possible_matches

  precisions = [0] * max_order
  for i in range(0, max_order):
    if smooth:
      precisions[i] = ((matches_by_order[i] + 1.) /
                       (possible_matches_by_order[i] + 1.))
    else:
      if possible_matches_by_order[i] > 0:
        precisions[i] = (float(matches_by_order[i]) /
                         possible_matches_by_order[i])
      else:
        precisions[i] = 0.0

  if min(precisions) > 0:
    p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
    geo_mean = math.exp(p_log_sum)
  else:
    geo_mean = 0

  ratio = float(translation_length) / reference_length

  if ratio > 1.0:
    bp = 1.
  else:
    bp = math.exp(1 - 1. / ratio)

  bleu = geo_mean * bp
  return (bleu, precisions, bp, ratio, translation_length, reference_length)

In [103]:
compute_bleu(layer1,list_of_lists_,1,True)

(0.32459051375236697,
 [0.32894736842105265],
 0.9867551618071956,
 0.9868421052631579,
 75,
 76)

In [97]:
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(layer1,list_of_lists_)
print(score)

0.18377079424627574


In [73]:
from nltk.translate.bleu_score import corpus_bleu
print(len(layer1))
print(len(list_of_lists_)-1)
score = corpus_bleu(layer1,list_of_lists_)
print(score)

19
18
0.18377079424627574


In [77]:
import nltk.translate.gleu_score as gleu
sc=gleu.corpus_gleu(layer1,list_of_lists_)
print("score: {}".format(sc))

score: 0.20642201834862386


In [48]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from itertools import chain, product
from nltk.translate.meteor_score import meteor_score

In [49]:
import numpy as np

def corpus_meteor(expected, predicted):
    meteor_score_sentences_list = list()
    [meteor_score_sentences_list.append(meteor_score(expect, predict)) for expect, predict in zip(expected, predicted)]
    meteor_score_res = np.mean(meteor_score_sentences_list)
    return meteor_score_res

In [ ]:
corpus_meteor()

In [85]:
b_file = open("reference.txt", "r")


reference= []

for line in b_file:
    stripped_line = line.strip()
    reference.append(stripped_line)


b_file.close()


print(reference)
print(len(reference))

['Good morning.', 'Where is the bank?', 'Look at me singing!', 'It is 9 in the morning.', 'I want food.', 'I am understanding.', 'I understood.', 'I am not understanding.', 'Where are you going today?', 'This iphone costs seventy thousand.', 'I am really woried because of rain.', 'Family does not help me.', 'I am sorry.', 'I am engineer.', 'Wait, I am thinking.', 'Please call ambulance.', 'No smoking please.', 'Will he go to school?', "I haven't had tea yet."]
19


In [86]:
b_file = open("predict.txt", "r")


model_out = []

for line in b_file:
    stripped_line = line.strip()
    model_out.append(stripped_line)


b_file.close()


print(model_out)
print(len(model_out))

['it is in the afternoon', 'where is the bank', 'look at me singing !', 'i am hurt', 'i want food', 'i understand', 'i do not understand', 'i was painter', 'where are you going today?', 'i like blue', 'i am really woried because of rain', 'i have to me', 'i am sorry', 'i am engineer', 'wait i am thinking', 'please call an ambulance', 'no smoking please', 'will he go to school', "i haven't had tea yet"]
19


In [78]:
pip install rouge

You should consider upgrading via the 'C:\Users\lenovo\anaconda3\python.exe -m pip install --upgrade pip' command.


In [79]:
from rouge import Rouge

In [80]:
rouge = Rouge()
rouge.get_scores(model_out[:len(model_out)-3], reference[:len(reference)-1], avg=True)

{'rouge-1': {'f': 0.8333333286904762,
  'p': 0.8333333333333333,
  'r': 0.8333333333333333},
 'rouge-2': {'f': 0.7976190429761905,
  'p': 0.7976190476190476,
  'r': 0.7976190476190476},
 'rouge-l': {'f': 0.8333333286904762,
  'p': 0.8333333333333333,
  'r': 0.8333333333333333}}

In [87]:
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'f': 0.41904761563843484,
  'p': 0.41441102756892234,
  'r': 0.4258145363408522},
 'rouge-2': {'f': 0.30451127530612254,
  'p': 0.3026315789473684,
  'r': 0.30701754385964913},
 'rouge-l': {'f': 0.41904761563843484,
  'p': 0.41441102756892234,
  'r': 0.4258145363408522}}